In [1]:
import os
if  not os.getcwd().endswith('juxtorpus'): 
    os.chdir('../')
assert os.getcwd().endswith('juxtorpus'), f"Working directory should be at juxtorpus. But at {os.getcwd()}"
f"Working directory: {os.getcwd()}" 

'Working directory: /Users/hcha9747/workspace/juxtorpus'

# Processing

This demos processing of the Corpus class using Processors.

In [2]:
from juxtorpus.corpus import CorpusBuilder
from pathlib import Path

builder = CorpusBuilder(Path("./assets/samples/elonmusk_tweets.csv"))
builder.head(5)

,Unnamed: 0,id,created_at,doc
0,0,849636868052275200,2017-04-05 14:56:29,And so the robots spared humanity ... https://...
1,1,848988730585096192,2017-04-03 20:01:01,@ForIn2020 @waltmossberg @mims @defcon_5 Exact...
2,2,848943072423497728,2017-04-03 16:59:35,"@waltmossberg @mims @defcon_5 Et tu, Walt?"
3,3,848935705057280001,2017-04-03 16:30:19,Stormy weather in Shortville ...
4,4,848416049573658624,2017-04-02 06:05:23,@DaveLeeBBC @verge Coal is dying due to nat ga...


In [3]:
builder.show_columns()

,0,1,2,3
All Columns,Unnamed: 0,id,created_at,doc
Added,,,,


In [4]:
builder.set_text_column('doc')
builder.set_nrows(100)
builder.add_meta('created_at', dtype='datetime')
corpus = builder.build()

corpus.summary()

Number of words           1386
Number of unique words     752
Number of documents        100
Name: frequency, dtype: uint64

### Process corpus with spaCy

In [5]:
import spacy, pathlib
from juxtorpus.corpus.processors import SpacyProcessor  # importing this module is required to register functions to global spacy registry.

nlp = spacy.load(pathlib.Path('/Users/hcha9747/Downloads/juxtorpus'))
nlp.factories.keys()

dict_keys(['attribute_ruler', 'tok2vec', 'merge_noun_chunks', 'merge_entities', 'merge_subtokens', 'token_splitter', 'doc_cleaner', 'parser', 'beam_parser', 'lemmatizer', 'trainable_lemmatizer', 'entity_linker', 'ner', 'beam_ner', 'entity_ruler', 'tagger', 'morphologizer', 'senter', 'sentencizer', 'textcat', 'spancat', 'future_entity_ruler', 'span_ruler', 'textcat_multilabel', 'stateless_custom_component', 'stateful_custom_component', 'extract_hashtags', 'en.lemmatizer'])

In [6]:
from juxtorpus.corpus.processors import SpacyProcessor

spacy_processor = SpacyProcessor(nlp)
corpus = spacy_processor.run(corpus)

doc = next(iter(corpus))
doc, type(doc)

(And so the robots spared humanity ... https://t.co/v7JUJQWfCv,
 spacy.tokens.doc.Doc)

In [7]:
corpus.history()

[<ProcessEpisode> Spacy Processor processed on 2022-09-29 15:08:05.599074 with pipeline components tok2vec, tagger, parser, attribute_ruler, lemmatizer, ner, extract_hashtags.]

In [8]:
corpus.metas().get('extract_hashtags').preview(5)

,text,extract_hashtags
0,And so the robots spared humanity ... https://...,[]
1,@ForIn2020 @waltmossberg @mims @defcon_5 Exact...,[]
2,"@waltmossberg @mims @defcon_5 Et tu, Walt?",[]
3,Stormy weather in Shortville ...,[]
4,@DaveLeeBBC @verge Coal is dying due to nat ga...,[]


In [9]:
type(next(iter(corpus)))

spacy.tokens.doc.Doc

In [10]:
from juxtorpus.corpus import CorpusSlicer

slicer = CorpusSlicer(corpus)
sliced = slicer.filter_by_item('extract_hashtags', '#elonmusk')

for doc in sliced:
    print(doc)

@jDaz Because I was a dumb idiot and didn't realize at the time that it would cause confusion #elonmusk
@varunfatehpuria Oh Model Y is coming too in a few years. Kinda have to. #elonmusk


In [11]:
sliced.history()

[<ProcessEpisode> Spacy Processor processed on 2022-09-29 15:08:05.599074 with pipeline components tok2vec, tagger, parser, attribute_ruler, lemmatizer, ner, extract_hashtags.]

In [12]:
sliced.texts()

76    (@jDaz, Because, I, was, a, dumb, idiot, and, ...
83    (@varunfatehpuria, Oh, Model, Y, is, coming, t...
Name: text, dtype: object

In [13]:
for doc in sliced:
    print(type(doc), doc.text)

<class 'spacy.tokens.doc.Doc'> @jDaz Because I was a dumb idiot and didn't realize at the time that it would cause confusion #elonmusk
<class 'spacy.tokens.doc.Doc'> @varunfatehpuria Oh Model Y is coming too in a few years. Kinda have to. #elonmusk
